In [1]:
import matplotlib
matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os
import umap
import parmap

import glob2
import numpy as np
from sklearn.decomposition import PCA

from shapely.geometry import Polygon
from shapely import geometry
from matplotlib.patches import Polygon
from scipy.spatial import ConvexHull


# visualize results module
from Visualize import *
from Visualize import PCA_Analysis
import pickle as pk

# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


# visualize results module
from ROI_visualize import *




Autosaving every 180 seconds


In [8]:
###############################################################
################# VISUALIZE PCA 2D MAPS #######################
###############################################################

# ***************** Fig 2B *********************

masktxt = np.int32(np.loadtxt('/media/cat/4TBSSD/yuki/IJ2/genericmask.txt'))
#print (masktxt)

mask = np.ones((128,128),'float32')
for k in range(masktxt.shape[0]):
    #print (masktxt[k][0],masktxt[k][1])
    mask[masktxt[k][0],masktxt[k][1]]=np.nan

# 
fig=plt.figure()
data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Feb5_30Hz/IJ2pm_Feb5_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95_spatial.npy')
#print (data.shape)
data = data.reshape(data.shape[0],128,128)
pca_loadings = []
for k in range(4):
    ax=plt.subplot(2,2,k+1)
    vmin = np.min(data[k])
    vmax = np.max(data[k])
    
    # 
    #print (k, vmin,vmax)
    temp = data[k]*mask
    pca_loadings.append(temp)
    plt.imshow(temp, vmin=vmin, vmax=vmax)
    plt.xticks([])
    plt.yticks([])
plt.show()

In [4]:
####################################################
####################################################
####################################################
root_dir = '/media/cat/4TBSSD/yuki/'
min_pixels = 50

#
masks, ids = cleanup_rois(root_dir, min_pixels)

#
mask, names = load_mask_and_names(root_dir)
  
#
rois = plot_rois(ids, masks, mask,names, min_pixels)




FINAL ROIS:  [ 15  21  29  36  43  50  57  64  71 136 150 164 186 198 249 255 261 268
 275] 19


In [31]:
rois = np.array(rois)
print (rois.shape)

pca_loadings = np.array(pca_loadings)
print (pca_loadings.shape)

pca_norm = []
quartile = 0.25
for k in range(pca_loadings.shape[0]):
    vmin = np.nanmin(pca_loadings[k])
    vmax = np.nanmax(pca_loadings[k])
    print (vmin,vmax)
    
    temp = pca_loadings[k]
    temp = (temp-np.nanmin(temp))/(np.nanmax(temp)-np.nanmin(temp))
    vmin = np.nanmin(temp)
    vmax = np.nanmax(temp)
    print (vmin,vmax)
    
    
    # 
    ctr=0
    fig=plt.figure()
    
    for r in range(rois.shape[0]):
        rr = rois[r]
        idx = np.where(rr==0)
        rr[idx]=np.nan
        temp2 = temp*rr
        ax=plt.subplot(6,6,ctr+1)
        plt.imshow(temp2, vmin=0, vmax=1.0)
        
        idx1 = np.where(np.logical_and(temp2>0.25, temp2<0.75))
        ctr+=1
        plt.title(str(r))
        
        #
        temp2[idx1]=np.nan
        ax=plt.subplot(6,6,ctr+1)
        plt.imshow(temp2, vmin=0, vmax=1.0)
        plt.title(str(r))
        ctr+=1
        s = np.nansum(temp2)
        print (s)
        
    #break
plt.show()
    

(19, 128, 128)
(4, 128, 128)
0.0011750261342643473 0.01903867290345668
0.0 1.0
39.82500861604954
132.36747717698645
16.943766621228416
58.0761842050108
338.0507378593124
6.06559059919799
107.47687798252427
215.40739987036022
1.997779634686848
0.0
73.0620120025016
0.7207542209369929
12.203975664317152
5.518059585792279
17.79022980773945
2.6224586806952566
0.4725246910659457
42.26246183696982


ValueError: num must be 1 <= num <= 36, not 37

In [2]:
####################################################
#### COMPUTE PCA AND UMAP AVERAGE SCATER PLOT ######
####################################################

# ******************** FIG 2D ***********************

data = np.load('/home/cat/concatenated_trials.npy')
#data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
#data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar3_30Hz/IJ2pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
data = np.load('/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
print ("raw data: ", data.shape)
data_all = data[:,:,:900]
data_all = data_all[:,:4,750:]
mean = data_all.mean(0)

print ("data_all: ", data_all.shape)
print ("Mean: ", mean.shape)
# 
data_in = []
mean2 = []
width = 5
for k in range(0, data_all.shape[2],width):
    data_in.append(data_all[:,:,k:k+width])
    mean2.append(mean[:,k:k+width])

mean2 = np.array(mean2)
data_in = np.array(data_in) 
print ("data in: ", data_in.shape)
print ("mean: ", mean2.shape)

#
mean2 = mean2.reshape(mean2.shape[0],-1)[:,None]
print ("mean: ", mean2.shape)


# 
data_in = data_in.reshape(data_in.shape[0], data_in.shape[1],-1)
print ("data in: ", data_in.shape)

d = []
for k in range(data_in.shape[0]):
    d.extend(data_in[k])

d.extend(mean2[:,0])
data_in = np.array(d)
print ("data in: ", data_in.shape)
# 


# 
X = data_in
print ("data input: ", X.shape)


#############################################
#############################################
#############################################
#
pca = PCA(n_components=2)
pca.fit(X)
p = pca.transform(X)
print ("Pca output: ", p.shape)

import umap
fit = umap.UMAP()
%time u = fit.fit_transform(X)
print ("Umap output: ", u.shape)


#######################################
########### PLOT RESULTS ##############
#######################################
# flatten data for PCA
t_flat = []

n_chunks = 30              # MANUAL SETTINGS
chunk_width = 10
n_trials = 62
cs = []
for k in range(n_chunks):
    cs.append(k*np.ones(n_trials)/n_chunks)
cs=np.hstack(cs)
#print (cs[:300])
    
clrs = plt.cm.viridis(cs)
print (clrs)

print("p :", p.shape)

# 
fig = plt.figure(figsize=(10,10))
cm = plt.cm.get_cmap('viridis')
ax=plt.subplot(211)

plt.scatter(p[:,0], p[:,1], c=clrs, s=25, edgecolor = 'black', alpha=.5)

#
plt.scatter(p[-n_chunks:,0], p[-n_chunks:,1], 
            c=np.arange(n_chunks), 
            s=150, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=.8)
    

ax=plt.subplot(212)
sc = plt.scatter(u[:,0], u[:,1], c=clrs, s=25, edgecolor = 'black', alpha=.5)
plt.scatter(u[-n_chunks:,0], u[-n_chunks:,1], 
            c=np.arange(n_chunks), 
            s=150, cmap=plt.cm.viridis,
            edgecolor = 'black', alpha=.8)


if False:
    plt.savefig('/home/cat/pca_single_trials.png',dpi=300)
    plt.close()
else:
    plt.show()
    

raw data:  (61, 7, 1801)
data_all:  (61, 4, 150)
Mean:  (4, 150)
data in:  (30, 61, 4, 5)
mean:  (30, 4, 5)
mean:  (30, 1, 20)
data in:  (30, 61, 20)
data in:  (1860, 20)
data input:  (1860, 20)
Pca output:  (1860, 2)
CPU times: user 1min 35s, sys: 4.84 s, total: 1min 40s
Wall time: 13.6 s
Umap output:  (1860, 2)
[[0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 ...
 [0.916242 0.896091 0.100717 1.      ]
 [0.916242 0.896091 0.100717 1.      ]
 [0.916242 0.896091 0.100717 1.      ]]
p : (1860, 2)


In [ ]:
###################################################
### GENERATE ALL DATA VS. PULLS IN PCA SPACE ######
###################################################

# ************** FIG 2E Convex hull analysis
from Visualize import (get_data_and_triggers, 
                       get_pca_object_and_all_points,
                       project_data_pca,
                       plot_pca_scatter,
                       get_convex_hull,
                       plot_convex_hull,
                       get_sessions)


root_dir = '/media/cat/4TBSSD/yuki/'

animal_ids = ['IA1','IA2','IA3','IJ1',"IJ2","AQ2"]
session_id = 'all'


# 
for animal_id in animal_ids:
    pa = PCA_Analysis()
    pa.root_dir = '/media/cat/4TBSSD/yuki/'
    pa.session_id = 'all'
    pa.animal_id = animal_id

    # 
    pa.n_pca = 10
    pa.sliding_window = 30    # how many frames to take into analysis window
    pa.n_frames = 300         # how many frames back in time to analyze: 

    
    # 
    sessions = get_sessions(pa.root_dir,
                            pa.animal_id,
                            pa.session_id)


    for session in sessions:
        pa.session = session
        print (" processing: ", session)
        fname_out = os.path.join(pa.root_dir,
                                  pa.animal_id,
                                  'tif_files',
                                  pa.session,
                                  pa.session+'_convex_hull.npz')
        if os.path.exists(fname_out):
            print ("     data laready processed")
            print ('')
            continue
            
        fname_triggers = os.path.join(pa.root_dir,
                                      pa.animal_id,
                                      'tif_files',
                                      pa.session,
                                      'blue_light_frame_triggers.npz')

        fname_data = os.path.join(pa.root_dir,
                                     pa.animal_id,
                                     'tif_files',
                                     pa.session,
                                     pa.session+ '_aligned_butterworth_0.1hz_6.0hz.npy')

        pa.X, pa.triggers = get_data_and_triggers(fname_triggers,
                                                  fname_data)
        
        if pa.X is None:
            print ("     missing data/blue light")
            print ('')
            print ('')
            continue
            
        if pa.triggers.shape[0]<5:
            print ("     not enough points : ", pa.triggers.shape )
            print ('')
            print ('')
            continue
            
        # make PCA plots
        pa.pca, pa.all_points = get_pca_object_and_all_points(pa)


        # FIG 2 E top
        pa.p_lever =  project_data_pca(pa)

        # skip plotting option
        # plot_pca_scatter(pa)
         
        # FIG 2E
        pa =  get_convex_hull(pa)

        # skip plotting option
        # plot_convex_hull(pa)
        print ('')


 processing:  IA1pm_Feb1_30Hz
 triggers:  (3,)
 data:  (40000, 128, 128)
(39130, 128, 128)
X:  (39130, 16384)
     not enough points :  (3,)


 processing:  IA1pm_Feb2_30Hz
     data laready processed

 processing:  IA1pm_Feb3_30Hz
     data laready processed

 processing:  IA1pm_Feb4_30Hz
     missing data/blue light


 processing:  IA1pm_Feb5_30Hz
     missing data/blue light


 processing:  IA1pm_Feb9_30Hz
     data laready processed

 processing:  IA1pm_Feb10_30Hz
     data laready processed

 processing:  IA1pm_Feb11_30Hz
     data laready processed

 processing:  IA1pm_Feb12_30Hz
     missing data/blue light


 processing:  IA1pm_Feb15_30Hz
     data laready processed

 processing:  IA1pm_Feb16_30Hz
     data laready processed

 processing:  IA1pm_Feb17_30Hz
     data laready processed

 processing:  IA1pm_Feb18_30Hz
     data laready processed

 processing:  IA1pm_Feb19_30Hz
     data laready processed

 processing:  IA1pm_Feb22_30Hz
 triggers:  (4,)
 data:  (40000, 128, 128)
(3

In [85]:
#############################
##### PLOT CONVEX HULL ######
#############################

# Fig 2 E

fname = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_convex_hull.npz'
d = np.load(fname, allow_pickle = True)

animal_id = 'IJ1'

pa = PCA_Analysis()
pa.root_dir = '/media/cat/4TBSSD/yuki/'
pa.session_id = 'Feb3_'

pa.n_pca = 10
pa.sliding_window = 30    # how many frames to take into analysis window
pa.n_frames = 300         # how many frames back in time to analyze: 

pa.animal_id = animal_id
pa.ratio_single = d['ratio_single'][::-1]
pa.ratio_cumsum = d['ratio_cumsum'][::-1]
pa.ratio_random_single = d['ratio_random_single'][::-1]
pa.ratio_random_cumulative = d['ratio_random_cumulative'][::-1]


pa.session = get_sessions(pa.root_dir,
                      pa.animal_id,
                      pa.session_id)[0]

print (" processing: ", pa.session)
fname_out = os.path.join(pa.root_dir,
                          pa.animal_id,
                          'tif_files',
                          pa.session,
                          pa.session+'_convex_hull.npz')



pa.X, pa.triggers = get_data_and_triggers(fname_triggers,
                                          fname_data)

if pa.X is None:
    print ("     missing data/blue light")
    print ('')
    print ('')

elif pa.triggers.shape[0]<5:
    print ("     not enough points : ", pa.triggers.shape )
    print ('')
    print ('')
else:
    # make PCA plots
    pa.pca, pa.all_points = get_pca_object_and_all_points(pa)


    # FIG 2 E top
    pa.p_lever =  project_data_pca(pa)


    # scatter plot option
    plot_pca_scatter(pa)

        
# plot_convex_hull(pa)

 processing:  IJ1pm_Feb3_30Hz


NameError: name 'fname_triggers' is not defined

In [92]:
##########################################################
#### GENERATE PCA DISTRIBUTIONS OF ALL BODY MOVEMENTS ####
##########################################################

#  Fig 3E 
pa = PCA_Analysis()
pa.root_dir = '/media/cat/4TBSSD/yuki/'
pa.n_pca = 10
pa.sliding_window = 30    # how many frames to take into analysis window
pa.n_frames = 1         # how many frames back in time to analyze: 

# 
pa.umap_flag = False
pa.convex_hull_flag = False

pa.frame_rate = 30

########################################
animal_ids = ['IA1','IA2','IA3','IJ1',"IJ2","AQ2"]
#animal_ids = ['IA2']
for animal_id in animal_ids:
    
    fname_out = os.path.join(pa.root_dir, animal_id,'tif_files',
                            'res_simplex.npy')

    if os.path.exists(fname_out)==False:
        pa.animal_id = animal_id
        session = 'all'

        pa.plot_all = True
        pa.plot_3D = False
        pa.plot_flag = False


        #########################
        # get the correct sesion name:
        sessions = get_sessions(pa.root_dir,
                                  pa.animal_id,
                                  session)

        #print ("sessions: ", sessions)
        # 
        res_simplex = pca_scatter_body_movements_fig3(pa, sessions)

        np.save(fname_out, res_simplex)
    
    
print (" COMPLETED: # OF SESSIONS: ", len(res_simplex))

 COMPLETED: # OF SESSIONS:  110


In [93]:
##################################################
### PLOT CONVEX HULLS FOR BODY PARTS VS LEVER ####
##################################################

# # Fig 3E
 
lefts = []
rights = []
levers = []
nTrials = []
animal_ids = ['IA1','IA2','IA3','IJ1',"IJ2","AQ2"]

for animal_id in animal_ids:

    #fig=plt.figure()

    # 
    fname_simplex = os.path.join(pa.root_dir, animal_id,'tif_files/res_simplex.npy')
    res_simplex = np.load(fname_simplex,allow_pickle=True)
    
    # get the correct sesion name:
    sessions = get_sessions(pa.root_dir,
                            animal_id,
                            session)

    (lever_vs_left_paw, 
     lever_vs_right_paw, 
     lever_vs_all, 
     n_trials) = plot_intersection_convex_hulls_lever_vs_bodyparts(res_simplex,
                                                                   sessions,
                                                                   animal_id,
                                                                   pa.root_dir)
    plt.suptitle(animal_id)

#     if False:
#         plt.savefig('/home/cat/polygons.png',dpi=300)
#         plt.close()
#     else:
#         plt.show()

    # 
    lefts.append(np.array(lever_vs_left_paw))
    rights.append(np.array(lever_vs_right_paw))
    levers.append(np.array(lever_vs_all))
    nTrials.append(np.array(n_trials))
#     # 
#     fig=plt.figure()
#     idx = np.where(np.isnan(left)==False)[0]
#     t = np.arange(len(left))[idx]
#     plt.scatter(t, left[idx])


#     # 
#     idx = np.where(np.isnan(right)==False)[0]
#     t = np.arange(len(right))[idx]
#     plt.scatter(t, right[idx])
#     plt.suptitle(animal_id)
plt.show()

all areas:  [1544226526.9314482, 1601138774.5727801, 1572212687.1700273, 1487279289.7549741, 1628631202.044745, nan]






all areas:  [1972198560.5312955, 1972903861.6629095, 1383075319.6999028, 1358264020.7434518, 1712588350.9203722, 535286015.17681813]
leftpaw lever % of region:  13.63 % % of all space:  3.88 %

rightpaw lever % of region:  10.68 % % of all space:  3.04 %





all areas:  [4366489304.783435, 4494376122.836153, 3336370560.2889113, 2681766132.25709, 3984054652.0013556, 423292644.95361394]
leftpaw lever % of region:  1.53 % % of all space:  0.48 %

rightpaw lever % of region:  0.0 % % of all space:  0.0 %





all areas:  [2117508237.4594607, 1994156968.2709773, 1028154423.1035535, 1668919053.7942195, 2007979406.0622149, nan]






all areas:  [3213651261.962522, 2871609429.8905025, 1806219733.9929695, 2980271306.054236, 2105522920.2287889, 805301597.359238]
leftpaw lever % of region:  12.39 % % of all space:  4.23 %

rightpaw lever % of region:  7.34 % % of all space:

all areas:  [1396232043.051174, 1164555116.1184125, 2395571556.5508776, 1866416937.9364529, 1666847585.7325168, 768996455.8592756]
leftpaw lever % of region:  0.53 % % of all space:  0.12 %

rightpaw lever % of region:  0.0 % % of all space:  0.0 %





all areas:  [3189415233.5676084, 2668195262.2125626, 2926095221.487699, 2539074325.416276, 2768395470.707589, 104269188.69042492]
leftpaw lever % of region:  3.27 % % of all space:  1.25 %

rightpaw lever % of region:  3.91 % % of all space:  1.25 %





all areas:  [3624653147.389246, 3255886832.3564034, 2770386950.843149, 3116884882.244893, 3475375949.1145344, 1070519365.0472969]
leftpaw lever % of region:  3.94 % % of all space:  1.22 %

rightpaw lever % of region:  5.57 % % of all space:  1.55 %





all areas:  [1330292146.712437, 1232607023.6918497, 1558080290.8945587, 1652917328.8564427, 1642235637.0562925, 441235502.1841447]
leftpaw lever % of region:  11.3 % % of all space:  2.48 %

rightpaw lever % of region:  14.31 % % of all

all areas:  [1146142931.7051713, 933519379.6498656, 932053403.6882893, 463301220.1250551, 890128703.163599, 583951934.9306276]
leftpaw lever % of region:  28.42 % % of all space:  4.41 %

rightpaw lever % of region:  25.22 % % of all space:  3.19 %





all areas:  [899605453.904674, 973779194.8258696, 1143150399.006774, 928747302.66403, 1056698752.5154889, nan]






all areas:  [1106172242.6979434, 1254107093.37608, 1162153674.5066268, 967027366.5715016, 1250321138.4836636, 618891031.1315807]
leftpaw lever % of region:  40.42 % % of all space:  6.34 %

rightpaw lever % of region:  36.4 % % of all space:  6.47 %





all areas:  [1170043565.8616345, 1134151360.135434, 1126288847.1389093, 938245939.3785572, 684711785.0902758, 410231317.78189147]
leftpaw lever % of region:  25.47 % % of all space:  3.75 %

rightpaw lever % of region:  27.58 % % of all space:  3.93 %





all areas:  [1304970509.1329486, 1211974860.9305804, 1241993242.140486, 649991329.4900042, 1172328338.7408645, 117998

In [110]:
##########################################
######## OVERLAP POLYGON RATIOS ##########
##########################################

# Fig 3J

from sklearn.linear_model import LinearRegression


def find_p_t(model, X, y, X2, y_pred):
    #self = super(LinearRegression, self).fit(X, y, n_jobs)
    #print (self)

    #print (y_pred.shape, y.shape)
    sse = np.sum((y_pred - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
    
    #print (sse)
    se = np.array([
        np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                for i in range(sse.shape[0])
                ])

    t = model.coef_ / se
    p = 2 * (1 - stats.t.cdf(np.abs(t), y.shape[0] - X.shape[1]))
    
    return t, p


clrs=['black','blue','red','green','magenta','pink']
sig = [0.05,0.01,0.001, 0.0001, 0.00001]
animal_ids = ['M1','M2','M3','M4','M5','M6']

fig=plt.figure()
for k in range(len(lefts)):
    temp = levers[k]
    
    # normalize to n trials per session
    if True:
        temp = temp/nTrials[k]
        
    print (temp.shape)
    
        
    idx = np.where(np.isnan(temp)==False)[0]
    #t=t[idx]
    temp=temp[idx]
    
    
    idx = np.where(temp!=0)[0]
    #t=t[idx]
    temp=temp[idx]
    
    
    model = LinearRegression()
    y=np.array(temp).reshape(-1, 1)
    x = np.arange(y.shape[0]).reshape(-1, 1)
    model.fit(x, y)

    x2 = np.arange(0,y.shape[0],1).reshape(-1, 1)
    y_pred = model.intercept_ + model.coef_ * x2


    t,p = find_p_t(model, x, y, x2, y_pred)
    
    temp_sig = ''
    for j in range(len(sig)):
        if p <= sig[j]:
            temp_sig = temp_sig+"*"
            
    
    plt.plot(x2, y_pred, label=animal_ids[k]+ " "+temp_sig,
             c=clrs[k],
            linewidth=6)
    
    
    
    t = np.arange(temp.shape[0])
    plt.scatter(t,temp,
                s=100,
                edgecolor='black',
               c=clrs[k],
               alpha=.7)

    
plt.legend(fontsize=12)
plt.xlim(0,43)
plt.ylim(bottom=0)
            
plt.show()

(24,)
(10,)
(11,)
(10,)
(9,)
(44,)


In [40]:
print (polygons[0])

[-34157.02254123 -31516.9013172  -25306.25891227 -18400.8155852
 -12575.85030282 -12310.53789496    872.33244449   5669.82166718
  18024.13495447  22738.45637924  41813.2832249   46402.41759252
  50166.65133501  52961.45630633]


In [139]:
import sphviewer as sph
from numpy.ma import masked_array

def myplot(x, y, nb=32, xsize=500, ysize=500):   
    xmin = np.min(x)
    xmax = np.max(x)
    ymin = np.min(y)
    ymax = np.max(y)

    x0 = (xmin+xmax)/2.
    y0 = (ymin+ymax)/2.

    pos = np.zeros([len(x),3])
    pos[:,0] = x
    pos[:,1] = y
    w = np.ones(len(x))

    P = sph.Particles(pos, w, nb=nb)
    S = sph.Scene(P)
    S.update_camera(r='infinity', x=x0, y=y0, z=0, 
                    xsize=xsize, ysize=ysize)
    R = sph.Render(S)
    R.set_logscale()
    img = R.get_image()
    extent = R.get_extent()
    for i, j in zip(range(4), [x0,x0,y0,y0]):
        extent[i] += j
    print (extent)
    
    
    clrs = np.unique(img)
    img_out = np.zeros(img.shape,'float32')
    for k in range(clrs.shape[0]):
        idx = np.where(img==clrs[k])
        print (k)
        img_out[idx]=k
    
    return img_out, extent

# Generate Lever locations
print (pa.p_lever.shape)
x = pa.p_lever.squeeze()[:,0]
y = pa.p_lever.squeeze()[:,1]
x_size = 100
y_size = 100
heatmap_lever, extent_lever = myplot(x,y, nb=4, xsize=x_size, ysize=y_size)

print ("npm ax: ", np.max(heatmap_lever), np.min(heatmap_lever))
print (np.unique(heatmap_lever))

v1_lever = masked_array(heatmap_lever,
                   heatmap_lever>=1)

# Generate left paw locations
print (pa_array[0].shape)
x = pa_array[0].squeeze()[:,0]
y = pa_array[0].squeeze()[:,1]
x_size = 100
y_size = 100
heatmap_leftpaw, extent_left_paw = myplot(x,y, nb=4, xsize=x_size, ysize=y_size)

print ("npm ax: ", np.max(heatmap_leftpaw), np.min(heatmap_leftpaw))
print (np.unique(heatmap_leftpaw))



fig,ax = plt.subplots()

#ax4.imshow(heatmap_64, 
ax1=plt.subplot(121)  
p1 = ax1.imshow(heatmap_lever,
               extent=extent_lever, 
               #origin='lower', 
               aspect='auto',
               cmap='Reds')

plt.xlim(-40000,60000)
plt.ylim(-50000,50000)

#cba = plt.colorbar(pa,shrink=0.25)

# 
ax2=plt.subplot(122)  
# v1_leftpaw = masked_array(heatmap_leftpaw,
#                           heatmap_leftpaw>=1)
p2 = ax2.imshow(heatmap_leftpaw, 
               extent=extent_left_paw, 
               #origin='lower', 
               aspect='auto',
               cmap='Greys')

plt.xlim(-40000,60000)
plt.ylim(-50000,50000)


#cbb = plt.colorbar(pb,shrink=0.25)
#plt.xlabel('Day')
#plt.ylabel('Depth')
#cba.set_label('positive')
#cbb.set_label('negative')
#plt.show()



plt.show()



(1, 31, 10)
[-41973.54    8432.656 -18250.133  34552.848]
0
1
2
npm ax:  2.0 0.0
[0. 1. 2.]
(1, 55, 10)
[-34157.016 265766.25  -61826.69   18024.137]
0
1
2
3
4
npm ax:  4.0 0.0
[0. 1. 2. 3. 4.]


In [37]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz_3secNoMove_movements.npz', allow_pickle=True)
features = data['feature_quiescent']
print (features.shape)
labels = data['labels']
print (labels)
temp = np.vstack(features[0])
print (temp[:10])
#plt.plot(features[0])


(7,)
['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
[[ 0.33333333  6.26666667]
 [ 8.33333333 11.46666667]
 [12.6        33.73333333]
 [37.06666667 51.33333333]
 [55.53333333 62.2       ]
 [63.46666667 75.66666667]
 [75.66666667 81.        ]
 [84.4        89.2       ]
 [89.33333333 93.33333333]
 [95.6        99.86666667]]


In [59]:
# # UMAP ON DATA
X_U = []
X_U.append(X_lever)
X_U.append(X_random)
X_U.append(X_shifted)
X_U = np.vstack(X_U)
print ("XU_:", X_U.shape)

import umap
umap_3d = umap.UMAP(n_components=3, init='random', random_state=0)

#fit = umap.UMAP()
%time umap_3d.fit(X_U[::10])
u = umap_3d.transform(X_U)

print ("Umap output: ", u.shape)


XU_: (339, 491520)
CPU times: user 1min 6s, sys: 1.09 s, total: 1min 7s
Wall time: 1min 7s
Umap output:  (339, 3)


In [61]:
fig=plt.figure()
#ax = plt.subplot(111)
ax = fig.add_subplot(projection='3d')
ax.scatter(
           u[:p_lever.shape[0],0], 
           u[:p_lever.shape[0],1],
           u[:p_lever.shape[0],2],
            c='red',
            s=100,
            edgecolor = 'black', alpha=1)

ax.scatter(
           u[p_lever.shape[0]:p_lever.shape[0]+p_random.shape[0],0], 
           u[p_lever.shape[0]:p_lever.shape[0]+p_random.shape[0],1], 
           u[p_lever.shape[0]:p_lever.shape[0]+p_random.shape[0],2], 
            c='black',
            s=100,
            edgecolor = 'black', alpha=1)

ax.scatter(
           u[p_lever.shape[0]+p_random.shape[0]:,0], 
           u[p_lever.shape[0]+p_random.shape[0]:,1], 
           u[p_lever.shape[0]+p_random.shape[0]:,2], 
    
            c='pink',
            s=100,
            edgecolor = 'black', alpha=1)
plt.show()